## SmoothQuant with `meta-llama/Meta-Llama-3-8B-Instruct`

In this tutorial, we will demonstrate how to apply SmoothQuant to Llama-3-8B instruct to quantize the weights and activations to int8.

In [3]:
%env CUDA_VISIBLE_DEVICES=6

env: CUDA_VISIBLE_DEVICES=6


## Install

Get started by installing SparseML via pip. You will need a GPU instance.

In [ ]:
!pip install sparseml[transformers]==1.8

### 1) Load Model

First, load a model from the Hugging Face hub (in this case `Meta-Llama-3-8B-Instruct`) using `SparseAutoModelForCausalLM`.

* `SparseAutoModelForCausalLM` is a wrapper around `AutoModelForCausalLM`, with some added utilities for saving and loading quantized models.

In [1]:
import torch
from sparseml.transformers import SparseAutoModelForCausalLM
from transformers import AutoTokenizer

MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

model = SparseAutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

ModuleNotFoundError: No module named 'torch'

### 2) Dataset

Next, load a dataset for calibrating the model. 

Best practices for calibration data:
* Apply the model's chat template to the sample data
* Use at least 512 samples. 1024 samples can improve the results sometimes
* Use at least 2048 sequence length. 4096 can improve the results sometimes
* Select a diverse, high quality dataset (ideally that is adapted to your use case)

In this case, we will use the [`HuggingFaceH4/ultrachat_200k` dataset](https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k), which contains multi-turn conversations and is generally a good choice for chat models.

In [2]:
from datasets import load_dataset

NUM_CALIBRATION_SAMPLES = 512

ds = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")
ds = ds.shuffle(seed=42).select(range(NUM_CALIBRATION_SAMPLES))

# Dataset should have "text" field with the data you want to use to calibrate
ds = ds.map(lambda batch: {
    "text": tokenizer.apply_chat_template(batch["messages"], tokenize=False)
})

Map: 100%|██████████| 512/512 [00:00<00:00, 4314.95 examples/s]


### 3) Recipe

Next, we make a recipe to specify the quantization algorithm to apply. 

XXX

In [3]:
recipe = """
quant_stage:
    quant_modifiers:
        SmoothQuantModifier:
            smoothing_strength: 0.5
            mappings: [
                [["re:.*q_proj", "re:.*k_proj", "re:.*v_proj"], "re:.*input_layernorm"],
                [["re:.*gate_proj", "re:.*up_proj"], "re:.*post_attention_layernorm"]
            ]
        GPTQModifier:
            sequential_update: false
            ignore: ["lm_head"]
            config_groups:
                group_0:
                    weights:
                        num_bits: 8
                        type: "int"
                        symmetric: true
                        strategy: "channel"
                    input_activations:
                        num_bits: 8
                        type: "int"
                        symmetric: true
                        strategy: "tensor"
                    targets: ["Linear"]
"""

### 4) Apply The Algorithm

After making the recipe, we can apply the quantization algorithm using the `oneshot` function.

> WARNING: You will need about 60GB of GPU RAM to run the below. To reduce memory consumption at the expense of speed, set `sequential_update: true` in your recipe.

In [4]:
from sparseml.transformers import oneshot

oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=2048,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

2024-06-12 02:29:14 sparseml.transformers.finetune.text_generation WARNING  Process rank: 0, device: cuda:0, n_gpu: 8, distributed training: True, 16-bits training: False
Logging all SparseML modifier-level logs to sparse_logs/12-06-2024_02.29.14.log
2024-06-12 02:29:14 sparseml.core.logger.logger INFO     Logging all SparseML modifier-level logs to sparse_logs/12-06-2024_02.29.14.log
Adding labels: 100%|██████████| 512/512 [00:00<00:00, 669.01 examples/s]
2024-06-12 02:29:16 sparseml.transformers.finetune.runner INFO     *** One Shot ***


['input_ids', 'attention_mask', 'labels']


2024-06-12 02:29:16 sparseml.core.recipe.recipe WARNING  Could not process input as a file path or zoo stub, attempting to process it as a string.
/home/rshaw/.pyenv/versions/3.10.14/envs/sparseml-env/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_fuse_fn_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/rshaw/.pyenv/versions/3.10.14/envs/sparseml-env/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_fuse_fn_kwargs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
2024-06-12 02:29:17 sparseml.modifiers.smoothquant.pytorch INFO     Running SmoothQuantModifier calibration with 512 samples...
100%|██████████| 512/512 [00:33<00:00, 15.22it/s]
2024-06-12 02:29:50 sparseml.modi

In [5]:
# Confirm generations of the quantized model look sane
input_ids = tokenizer("Hello my name is", return_tensors="pt").input_ids.to("cuda")
output = model.generate(input_ids, max_new_tokens=100)
print(tokenizer.decode(output[0]))

<s> Hello my name is John and my email is john@example.com I am interested in Hello my name is John and my email is john@example.com I am interested in Hello my name is John and my email is john@example.com I am interested in Hello my name is John and my email is john@example.com I am interested in Hello my name is John and my email is john@example.com I am interested in Hello my name is John and my email is


### 5) Serialize the model

Save the model using `save_pretrained` using `save_compressed=True`. This will save the weights in a compressed format, compatible for loading with vLLM!

In [ ]:
OUTPUT_DIR = "llama-3-gptq-4-bit"
model.save_pretrained(OUTPUT_DIR, save_compressed=True)
tokenizer.save_pretrained(OUTPUT_DIR)